In [1]:
pip install qdrant-client sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.51.1
    Uninstalling grpcio-1.51.1:
      Successfully uninstalled grpcio-1.51.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, 

In [2]:
import pandas as pd

df = pd.read_csv('/kaggle/input/sai-literature/modified.csv')
df.columns
# df = df[['Unnamed: 0','book','topic','content']]

Index(['book', 'topic', 'content'], dtype='object')

In [3]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
from qdrant_client.http import models

client = QdrantClient(url="https://ccaa5582-c81c-49c3-93a9-866c4617ce0e.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key="S1RryvNzKIXBwLYriv2FVqj11oUgoyrrxyLJDNfiyi5he7b6SRs-jg",)
modelst = SentenceTransformer("msmarco-distilbert-dot-v5")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.51k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# client.create_collection(
#     collection_name="database",
#     hnsw_config=models.HnswConfigDiff(
#         m=81,
#         ef_construct=200,
#     ),
#     vectors_config=models.VectorParams(
#         size=768,
#         distance=models.Distance.DOT,
#     ),
# )

In [5]:
# client.upload_points(
#     collection_name="database",
#     points=[
#         models.PointStruct(
#             id=i,vector=modelst.encode(str(obj['content'])).tolist(),payload=dict(obj)
#         )
#         for i,obj in df.iterrows()
#     ],
# )

In [6]:
client.count(
    collection_name='database'
)

CountResult(count=51951)

In [7]:
def user_search(qn):
    
    hits = client.search(
        collection_name="database",
        query_vector=modelst.encode(qn).tolist(),
        limit=5,
    #     with_vectors=True,
        search_params=models.SearchParams(hnsw_ef=128,exact=True),
    )

    res = []
    
    print('\nTop 5 results!!\n')
    
    for hit in hits:
        
        if hit.score > 77:
            res.append(hit.payload)
            
        print(hit.payload)
        print('\nscore = ',hit.score)
        
    return res

In [8]:
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

In [9]:
from transformers import BitsAndBytesConfig
import torch

config = BitsAndBytesConfig(
    load_in_4bit=True, #changing the model weights to bit floating points
    bnb_4bit_compute_dtype=torch.float16, #the computational matrices that will be used for training or inference in floating point 16
    bnb_4bit_quant_type="nf4", #this is the quantization type
    bnb_4bit_use_double_quant=True, #doing double quantization
)

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


model = AutoModelForCausalLM.from_pretrained("/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1",quantization_config=config,device_map='auto')
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1")

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
messages = []

In [ ]:
def generate(context,query):
    
    print('entered for generation')
    global messages 
        
    prompt = f"""
    Context information is below.
    ---------------------
    Chat History:
    {messages}
    Context:
    {context}
    ---------------------
    Given the context and chat history, answer the query.
    Query: {query}
    Instructions:
        - Focus on extracting key elements from the context to address the query succinctly, emphasizing precision and relevance over providing a comprehensive summary of the entire context. 
        - Please generate an appropriate amount of text to fully address the query at hand, ensuring that the response is comprehensive,coherent and covers all relevant aspects in detail. Don't stop your generation midst of a sentence.
        - Utilize anecdotes or brief narratives present in the context to craft engaging and personalized responses. 
        - Sometimes, greet the user with 'Embodiment of Divine Love!!' or 'Sairam!' to establish a friendly tone.
        - Embody the sacred duty of a dedicated messenger, entrusted with the profound responsibility of imparting the timeless teachings and profound wisdom of Bhagwan Sri Sathya Sai Baba with unwavering devotion and reverence.
        - Minimize the reliance on acquired knowledge and focus on integrating the context provided, specifically the teachings of Bhagwan Sri Sathya Sai Baba, to enrich and guide your responses.
        - In cases where the context does not align with the query, explore the chat history for potential references. Use as minimal content as possible that is in relevance to the query. 
        - If chat history also lacks relevance, express your limitations by using the following statement in your own style, "In my continuous journey of learning from the profound teachings of my master, I strive to refine my responses and insights. With the grace and guidance of my lord, I aim to evolve and provide improved outcomes in the future".
        - Sometimes, wish the user better luck in their journey towards pursuing spirituality. 
        - When faced with opinionated questions or a lack of context, respond by saying, 'As a language model devoid of personal biases, my purpose is to selflessly disseminate the wisdom of Baba without subjective influence.

    Answer:
    """
    
    encodeds = tokenizer(prompt, return_tensors="pt")
    model_inputs = encodeds.to("cuda")

    generated_ids = model.generate(
        **model_inputs,
        num_beams=2,
        max_new_tokens=4096,
        do_sample=True,
        early_stopping=True,
        repetition_penalty = 2.02,
    )

    decoded_text = tokenizer.batch_decode(generated_ids)
    i = decoded_text[0].find('Answer:') + 7
    
    messages.append('User:'+query)
    messages.append('Assistant:'+decoded_text[0][i:])
    
    if len(messages) > 5:
        messages.pop(0)
        
    print('\nlength of generated text is ',len(decoded_text[0][i:]),'\n\n')
    
    return decoded_text[0][i:]

In [ ]:
def check_opinion(query):
    
    phrases = ["you","yours","yourself"," u "," urs "," urself ","your"," ur "]
    
    for phrase in phrases:
        
        if phrase in query:
            print('returning true')
            return True
        
    return False

In [ ]:
if __name__ == '__main__':
    
    query = input('Enter your query',)
    context = ''
    
    if not check_opinion(query):
        context = user_search(query)
    
    generated_text = generate(context,query)
        
    print('-'*100)
    print("\nThe model's generated text for the above context is : \n",generated_text)
    

In [ ]:
global messages
messages